In [ ]:
from datasets import load_dataset
from model import NERClassifier
from preprocess_dataset import NERDataset 
from trainner import Trainner
from transformers import BertTokenizer
from preprocess_dataset import remove_empty_entries

import torch

## Setups

In [ ]:
CHECKPOINT = 'neuralmind/bert-base-portuguese-cased'
tokenizer = BertTokenizer.from_pretrained(CHECKPOINT)

In [ ]:
data = "lener_br"
dataset = load_dataset(data)
dataset = remove_empty_entries(dataset)

In [ ]:
MAX_LEN = 256
n_labels = 13 #TODO: por enquanto hardcodado 

In [ ]:
pytorch_dataset_train = NERDataset(data=dataset['train'], max_len=MAX_LEN, tokenizer=tokenizer)
teste = pytorch_dataset_train[0]

In [ ]:
model = NERClassifier(n_labels=n_labels, checkpoint=CHECKPOINT)

## Passando dados pelo modelo

In [ ]:
input_ids = teste['input_ids']
attention_mask = teste['attention_mask']
labels = teste['targets']

In [ ]:
out = model(input_ids, attention_mask, labels=labels)

In [ ]:
print(out)

## Explorando um pouco o dataset

In [ ]:
type(dataset)

In [ ]:
dataset

In [ ]:
max_vals = [torch.max(val['targets']) for val in pytorch_dataset_train]

In [ ]:
interval = (min(max_vals).item(), max(max_vals).item()

## Testando o DataLoader

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
batch_size = 8

In [ ]:
pytorch_dataset_train

In [ ]:
teste = iter(DataLoader(pytorch_dataset_train ,batch_size=batch_size))

In [ ]:
mini_batch = next(teste)

## Passando dados pelo modelo em batch

In [ ]:
ids = mini_batch['input_ids'].squeeze().to("cuda", dtype = torch.long) # Remove a dimensão 1 ali do meio, talvez funcione?
att = mini_batch['attention_mask'].to("cuda", dtype = torch.long)
targets = mini_batch['targets'].to("cuda", dtype = torch.long)

In [ ]:
print(ids.shape)

In [ ]:
model = model.to("cuda")

In [ ]:
outs = model(ids, att, labels=targets)

In [ ]:
outs

## Convertendo predições para etiquetas

Ja vimos como converter um output de batch_size=1 para labels. Agora vamos tentar pra um batch inteiro

### TODO
- [ ] Mover a lógica de Sigmoid pra dentro do batch_to_labels

In [ ]:
from utils import batch_to_labels, preds_to_label_idx

In [ ]:
from torch.nn import Sigmoid

In [ ]:
s = Sigmoid()

In [ ]:
b_logits = outs.logits
probs = s(b_logits)

In [ ]:
preds_to_label_idx(probs[0])

In [ ]:
outputs = batch_to_labels(b_logits)

In [ ]:
outputs